#Importing Library

In [15]:
import spacy
from spacy import displacy
import ramdom
from spacy.util import minibatch,compounding

In [16]:
nlp=spacy.load('en_core_web_sm')
nlp.pipe_names

['tagger', 'parser', 'ner']

In [17]:
ner=nlp.get_pipe('ner')

#Creating a custom dataset

In [18]:
TRAIN_DATA=[
            ("I want to buy new shoes from amazon at cheap price.",
                  {"entities": [(18, 23, "PRODUCT"),
                                (29, 35, "BRAND"),
                                (39, 44, "SERVICE")]}),
                 ("I am looking for brand new expensive near Noida",
                  {"entities": [(27, 33, "PRODUCT"), (34, 43, "SERVICE"),(43,53,"LOCATION")]}),
                 ("I want to replace my watch at affordable price near Sarojini Nagar",
                  {"entities": [(21, 26, "PRODUCT"), (30, 40, "SERVICE"),(46,65,"LOCATION")]}),
                 ("I am looking for some groceries at cheap price",
                  {"entities": [(22, 31, "PRODUCT"),
                                (35, 40, "SERVICE")]}),
                 ("I want to buy a used car",
                  {"entities":[(21,23,"PRODUCT")]}),
                 ("some product are cheap some are expensive some are affordable near Noida",
                  {"entities":[(17,22,"SERVICE"),(32,41,"SERVICE"),(51,60,"SERVICE"),(61,71,"LOCATION")]}),
                 ("Popular locations in Noida Sector 10",
                  {"entities":[(17,25,"LOCATION")]}),
                 ("good adidas shoes at cheap price",
                  {"entities":[(5,11,"BRAND"),(12,17,"PRODUCT"),(21,26,"SERVICE")]}),
                 ("i would love to have a pizza right now",
                  {"entities":[(23,28,"PRODUCT")]}),
                 ("Is there any affordable bike service near me",
                  {"entities":[(13,23,"SERVICE"),(24,28,"PRODUCT")]}),
                 ("Where i can buy a furniture",
                  {"entities":[(18,26,"PRODUCT")]}),
                 ("where i can buy a good home furniture at cheap price",
                  {"entities":[(28,37,"PRODUCT"),(41,46,"SERVICE")]}),
                 ("any laptop seller near kalkaji",
                  {"entities":[(4,10,"PRODUCT"),(17,29,"LOCATION")]}),
                 ("any hospital near mathura",
                  {"entities":[(12,24,"LOCATION")]}),
                 ("i want to go to party",
                  {"entities":[(16,20,"LOCATION")]}),
                 ('I have lost my wallet need to lodge a complaint.',
                  {'entities':[(15,21,"PRODUCT")]}),
                 ("affordable furniture at amazon",
                  {'entities':[(0,9,"SERVICE"),(10,19,"PRODUCT"),(23,29,"BRAND")]}),
                 ("Can i get some good product near Delhi",
                  {'entities':[(27,37,"LOCATION")]}),
                 ("expensive product near Noida",
                  {'entities':[(27,37,"LOCATION")]}),
                 ("is there any amazon shop in Noida",
                  {'entities':[(12,18,"BRAND"),(24,32,"LOCATION")]}),
                 ("any affordable shop in Noida",
                  {'entities':[(3,13,"SERVICE"),(19,27,"LOCATION")]}),
                 ("i am looking for a cheap shoes that i can find near to Noida so i dont have to go to amazon",
                  {'entities':[(18,23,"SERVICE"),(24,29,"PRODUCT"),(46,59,"LOCATION")]})
                 
]

**Now here i have created a custom dataset according to the mail because on internet i was not able to find this type of dataset that can relate this work**

In [19]:
for _,annotation in TRAIN_DATA:
  for ent in annotation.get('entities'):
    ner.add_label(ent[2])

In [20]:
pipe_want=['ner','trf_wordpiecer','trf_tok2vec']
use_pipe=[pipe for pipe in nlp.pipe_names if pipe not in pipe_want]

#Training model

In [ ]:
with nlp.disable_pipes(*use_pipe):
  for interation in range(30):
    random.shuffle(TRAIN_DATA)
    losse={}
    batches=minibatch(TRAIN_DATA,size=compounding(4.0,32.0,1.001))
    for batch in batches:
      texts,annotation=zip(*batch)
      nlp.update(texts,annotation,
                 drop=0.25,
                 losses=losse)

In [26]:
doc=nlp('is ther any coffe shop near Noida')
for ent in doc.ents:
  print(ent.text,ent.label_)

coffe PRODUCT
Noida GPE


In [30]:
docs=nlp('where i can get affordable furniture shop')
for ent in docs.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)
displacy.render(nlp(docs.text),style='ent',jupyter=True)

furniture 27 36 PRODUCT


**As i have a small dataset i am getting normal aacuracy but if i might have a good dataset i could get a better accuracy and for that we can use the chats options that we have in the company or we can generate api to get the dataset**

In [33]:
docs=nlp('i want a cheap shoe or any type of furniture near Delhi')
for ent in docs.ents:
  print(ent.text,ent.start_char,ent.end_char,ent.label_)
displacy.render(nlp(str(docs.text)),style='ent',jupyter=True)

Delhi 50 55 GPE


In [34]:
import re
expression=r"near(.*)$"
for match in re.finditer(expression,docs.text):
  start,end=match.span()
  span=docs.char_span(start,end)
  if span is not None:
    print("Location",span[1].text)

Location Delhi


**With the help of this regex we can extract the location which are like near to or in or around**